In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install llama-index

In [ ]:
import llama_index
import pandas as pd
df=pd.read_csv('/kaggle/input/ecommerce-rag/all_products.csv')


In [ ]:
from llama_index.core import Document

documents = [
    Document(
        content=f"{row['product_title']} {row['product_bullet_point']} {row['product_brand']}",
        metadata={
            'ID': row['product_id'],
            'Title': row['product_title']
        }
    )
    for _, row in df.iterrows()
]

In [ ]:
!pip install transformers
!pip install llama-index-embeddings-huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load a BERT model for embeddings
bert_model_id = "BAAI/bge-small-en-v1.5"  # Example BERT model for embeddings
tokenizer0 = AutoTokenizer.from_pretrained(bert_model_id)
bert_model = AutoModel.from_pretrained(bert_model_id)

# Create the HuggingFaceEmbedding instance for embeddings using BERT
embed_model = HuggingFaceEmbedding(model_name=bert_model_id)

# Optionally set this embedding model in Settings
from llama_index.core import Settings
Settings.embed_model = embed_model

In [ ]:
'''from llama_index.core import VectorStoreIndex

# Initialize the main index with an empty vector store or the appropriate vector store for your use case
main_index = VectorStoreIndex(documents)


# Set batch size based on memory and performance constraints
batch_size = 100

# Iterate over the documents in batches
for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    # Create a partial index for this batch
    partial_index = VectorStoreIndex.from_documents(batch, embedding_model)
    # Merge the partial index into the main index
    main_index.merge(partial_index)

# Now main_index contains all documents indexed in batches'''


In [ ]:
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex.from_documents(documents,show_progress=True)

In [ ]:
!pip install pickle

In [ ]:
import pickle

with open("vector_store_index.pkl", "wb") as file:
    pickle.dump(index, file)



In [ ]:
import pickle

with open("vector_store_index.pkl", "rb") as file:
    index = pickle.load(file)


In [ ]:
index

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
retriever=VectorIndexRetriever(index=index,similarity_top_k=4)

In [ ]:
query = "pull-up station"
top_k = 5  # Number of results you want to retrieve

# Retrieve documents similar to the query
results = retriever.retrieve(query)




In [ ]:
results[0]

In [ ]:
for result in results:
    print(result.metadata['ID'],result.metadata['Title'])

In [ ]:
documents[11]